In [1]:
cs = [['.', 'x', '.', '.', '.'], 
      ['x', 'o', '.', '.', '.'], 
      ['x', 'o', '.', '.', '.'],
      ['x', 'o', '.', '.', 'o'],
      ['.', '.', '.', '.', 'o']]

In [2]:
class Node:
    def __init__(self, state, parent):
        self.state = state
        self.parent = parent
    
    def get_state(self):
        return self.state
    
    def get_parent(self):
        return self.parent

In [3]:
def get_list_point(state, turn):
    '''
        state: [[]]
        turn: string, 'player' or 'bot'
        return: list of coordinate
    '''
    ls_point = list()
    form_point = {'player': 'o', 'bot': 'x'}
    for row in range(state.__len__()):
        for cell in range(state.__len__()):
            if state[row][cell] == form_point[turn]:
                ls_point.append(str(row) + '-' + str(cell))
    return ls_point

In [4]:
def forward(ls_point, num_point):
    '''
        ls_point: list()
        num_point: list()
        return: new num_point if next point can make a line with previous
    '''
    x_src, y_src = list(map(int,num_point[-1].split('-')))
    x_pre, y_pre = list(map(int,num_point[-2].split('-')))
    x_next, y_next = x_src + (x_src - x_pre), y_src + (y_src - y_pre)
    if str(x_next) + '-' + str(y_next) in ls_point:
        num_point.append(str(x_next) + '-' + str(y_next))
        return forward(ls_point, num_point)

In [5]:
def near(point_src, point_des):
    '''
        point_src: str, coordinate of source point, 'x-y'
        point_des: str, coordinate of destination point, 'x-y'
        return: boolean, True if it close together and False if not
    '''
    x_src, y_src = list(map(int,point_src.split('-')))
    x_des, y_des = list(map(int,point_des.split('-')))
    return abs(x_src - x_des) <= 1 and abs(y_src - y_des) <= 1

In [6]:
def count_line(state, score, turn):
    '''
        score: int, number of point in line that current turn want to find
        turn: string, 'player' or 'bot'
        return: number of line such that point >= score
    '''
    ls_point = get_list_point(state, turn)
    if score == 1:
        return ls_point.__len__()
    num_line = 0
    for point_src in ls_point[:-1]:
        num_point = [point_src, ]
        for point_des in ls_point[ls_point.index(point_src) + 1:]:
            if near(point_src, point_des):
                num_point.append(point_des)
                forward(ls_point, num_point)
            else:
                continue
            if num_point.__len__() >= score:
                num_line += 1
            num_point = [point_src, ]
    return num_line

In [7]:
def count_score(state, turn):
    '''
        state: [[]]
        turn: string, 'bot' or player
        return: int, score for each line contain more than 2 points
    '''
#     state = state.get_value()
    table_score = {5: 200, 4:50, 3:10, 2:2}
    total_score = 0
    for score in table_score:
        total_score += count_line(state, score, turn)*table_score[score]
    return total_score

In [8]:
def profit(state, turn):
    '''
        state: a Node
        turn: player or bot
        return: int (value of profit)
    '''
    score_make_line = count_score(state, turn)
#     score_in_opp_line = count_point_in_opp_line(state, turn)
#     score_opp_in_line = count_opp_in_line(state, turn)
    return score_make_line #+ score_in_opp_line - score_opp_in_line

In [9]:
def can_move(state, turn):
    '''
        state: current state of game
        return: list of new state
    '''
    pass

In [10]:
def get_max(crr_state, parent_value):
    '''
        state: a Node
        return: a Node with max profit
    '''
    best_node = Node(None, None)
    max_profit = - 10000
    for move in can_move(crr_state, 'bot'):
        if parent_value < profit(move, 'bot') and parent_value != -10000:
            return [False, None, None]
        
        if max_profit < profit(move, 'bot'):
            max_profit = profit(move, 'bot')
            best_node = move
    return [True, max_profit, best_node]

In [11]:
def get_min(crr_state, parent_value):
    '''
        state: a Node
        return: a Node with min profit
    '''
    best_node = Node(None, None)
    min_profit = 10000
    for move in can_move(crr_state, 'player'):
        if parent_value > profit(move, 'player') and parent_value != 10000:
            return [False, None, None]
        
        if min_profit > profit(move, 'player'):
            min_profit = profit(move, 'player')
            best_node = move
    return [True, min_profit, best_node]

In [12]:
def mini_max(crr_state, deepth, get_max_profit, parent_value):
    '''
        state: a Node, current state of game
        crr_path: list(), the predict move of both player
        deepth: int, high of tree
        return: new state
        
        - Calculate profit(state) in list of can_move(state)
        - Best move is min or max profit(state) depend of deep value
    '''
    if deepth == 1:
        if get_max_profit:
            return get_max(crr_state, parent_value)
        return get_min(crr_state, parent_value)
    else:
        if get_max_profit:
            best_node = Node(None, None)
            parent_value = - parent_value
            for move in can_move(crr_state, 'bot'):
                get_value = mini_max(move, deepth - 1, not get_max_profit, parent_value)
                
                if not get_value[0]:
                    continue
                
                if get_value[1] > parent_value:
                    parent_value = get_value[1]
                    best_node = move
            return [True, parent_value, best_node]
        else:
            best_node = Node(None, None)
            parent_value = - parent_value
            for move in can_move(crr_state, 'player'):
                get_value = mini_max(move, deepth - 1, not get_max_profit, parent_value)
                
                if not get_value[0]:
                    continue
                
                if get_value[1] < parent_value:
                    parent_value = get_value[1]
                    best_node = move
            return [True, parent_value, best_node]


In [13]:
def bot_turn(crr_state, deepth):
    '''
        crr_state: a Node
        return: new state (best move get in minimax(state))
    '''
    display(mini_max(crr_state, deepth, True, 10000)[2])

In [14]:
# bot_turn(cs, 1)

TypeError: 'NoneType' object is not iterable

In [ ]:
def player(state):
    '''
        state: current state of game
        return: new state with that move
    '''
    pass

In [ ]:
def finish(state):
    '''
        state: current state of game
        return: True, winner (winner: bot or player)
        - True: 5 point in line
    '''

In [ ]:
def display(state):
    '''
        print(state)
    '''
#     state = cs # state.get_value()
    for i in state:
        for j in i:
            print(j, end=' ')
        print()
    pass

In [ ]:
def caro_game():
    '''
        declare: deep
    '''
    deep = input('Enter the level of game: ')
    state = load_game()
    while not finish(state)[0]:
        state = bot_turn(state, deep)
        display(state)
        state = player_turn(state)
        display(state)
    if finish(state)[1] == 'player':
#         print('You win!')
        pass
    else:
#         print('Bot win!')
        pass
    pass